In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [3]:
import sys
sys.path.insert(0, "../code-previous")
sys.path.insert(0, "../tagger-scrap")

sys.path.insert(0, "..")

In [4]:
import os
from nltk.tag.stanford import StanfordPOSTagger
java_path = "/home/sreekumar_s/jdk1.8.0_131/bin/java"
os.environ['JAVAHOME'] = java_path


import CMUTweetTagger

In [5]:
path_to_model = "../pos-tagger-models/english-bidirectional-distsim.tagger"
path_to_jar = "../pos-tagger-models/stanford-postagger.jar"
tagger = StanfordPOSTagger(path_to_model, path_to_jar)
tagger.java_options='-mx4096m'

In [6]:
import pandas as pd
import numpy as np
import nltk
stopwords = nltk.corpus.stopwords.words('english')

import plotly

import swifter
import dask.dataframe as dd
from dask.multiprocessing import get

from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set()
import mutual_information
import matplotlib.pyplot as plt
path2rawdata = '/mnt/disks/vault/wos2017-parsed/'

In [7]:
raw_data_path = "/mnt/disks/vault/analysis-data/nature-science-data-full/nature_science_journal_data.pql"
data = pd.read_pickle(raw_data_path)

data = data[(data.Journal == "NATURE") & (data.PubYear_x > 1970) & (data.PubYear_x < 1981)]

In [8]:
data.PubYear_x.unique()

array([1980, 1978, 1971, 1974, 1979, 1976, 1977, 1972, 1973, 1975])

In [9]:
len(data)

18245

In [10]:
data.head()

,ArticleID,PubYear_x,Doctypes,Title,Journal,Journal Abbreviation ISO,PubYear_y,c5
41958,A1980JB44400046,1980,Article,COMPACT MYELIN EXISTS IN THE ABSENCE OF BASIC-PROTEIN IN THE SHIVERER MUTANT MOUSE,NATURE,Nature,1980,32.0
41959,A1980KD36500043,1980,Article,ASYMMETRIC-FC RECEPTOR DISTRIBUTION ON HUMAN-PMN ORIENTED IN A CHEMOTACTIC GRADIENT,NATURE,Nature,1980,22.0
41960,A1980JK26000020,1980,Letter,DISPOSAL OF CARCINOGENS,NATURE,Nature,1980,3.0
41962,A1980KD36500050,1980,Article,MONOCLONAL ANTI-ACETYLCHOLINE RECEPTOR ANTIBODIES CAN CAUSE EXPERIMENTAL MYASTHENIA,NATURE,Nature,1980,38.0
41963,A1980KM41100042,1980,Article,"EXCRETION, ISOLATION AND STRUCTURE OF A NEW PHENOLIC CONSTITUENT OF FEMALE URINE",NATURE,Nature,1980,28.0


__Add tagging from the two parsers__

In [ ]:
data['stanford_tagger_tags'] = data['Title'].swifter.apply(lambda x: tagger.tag(x.split()))

Pandas Apply:   0%|          | 54/18245 [01:24<7:11:26,  1.42s/it] 

In [ ]:
import pickle
data.to_pickle("/mnt/disks/vault/analysis-data/stanford_tagger_data/pos_tagged_stanford_1971_1980.pql")

In [ ]:
#ark_tagger_outputs = CMUTweetTagger.runtagger_parse(list(data['Title']))
#data['ark_tagger_tags'] = pd.Series(ark_tagger_outputs)

In [ ]:
## https://stackoverflow.com/questions/45545110/how-do-you-parallelize-apply-on-pandas-dataframes-making-use-of-all-cores-on-o

ddata = dd.from_pandas(data, npartitions=30)

res = ddata.map_partitions(lambda df: df['Title'].apply((lambda x: tag(x.split())), axis=1)).compute(get=get)